 # Description of the notebook

 In this notebook I am downloaded shared data to my workspace.

 A dedicated API service was created for this workshop, available at: https://api-datalab.coderslab.com/api/v2. Additionally, documentation has been made available, which can be found here: [click](https://api-datalab.coderslab.com/v2/docs/).

According to the documentation, 4 endpoints have been made available:
 - `airport` - data about the airport,
 - `weather` - information about the recorded weather at the airport on a given day,
 - `aircraft` - data about aircraft
 - `flights` - data about departures from a given airport per day.

In order to retrieve information where the `airportId` parameter is required, I will used the list from the `airports.csv` file.

Token: `iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk`.

 ### Comments
 - API call limit is 1000/min

In [19]:
# import libraries
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
import os
from pandas import json_normalize

In [12]:
# API parameters
api_key = "API_KEY_TO_BE_REPLACED"
url = f'https://api-datalab.coderslab.com/api/v2'
headers = {"authorization": api_key}

In [15]:
# import airports.csv file and adaptation to further steps for downloading from endpoints
airports_df = pd.read_csv('./data/airports.csv')
airports_df
airport_ids = airports_df['origin_airport_id'].tolist()

In [16]:
# downloads airports data
def fetch_airport_data():
    airport_data = []
    for airport_id in airport_ids:
        response = requests.get(f"{url}/airport/{airport_id}", headers=headers)
        if response.status_code == 200:
            airport_data.append(response.json())
        time.sleep(0.1)  
    return pd.DataFrame.from_records(airport_data)

airport_df = fetch_airport_data()

In [17]:
airport_df

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,11638,Fresno Air Terminal,"Fresno, CA","FRESNO YOSEMITE INTERNATIONAL, CA US"
1,13342,General Mitchell Field,"Milwaukee, WI","MILWAUKEE MITCHELL AIRPORT, WI US"
2,13244,Memphis International,"Memphis, TN","MEMPHIS INTERNATIONAL AIRPORT, TN US"
3,15096,Syracuse Hancock International,"Syracuse, NY","SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US"
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
...,...,...,...,...
92,13198,Kansas City International,"Kansas City, MO","KANSAS CITY INTERNATIONAL AIRPORT, MO US"
93,10423,Austin - Bergstrom International,"Austin, TX","AUSTIN BERGSTROM INTERNATIONAL AIRPORT, TX US"
94,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
95,13303,Miami International,"Miami, FL","MIAMI INTERNATIONAL AIRPORT, FL US"


In [9]:
api_data_path = "./data/apidata"
isExist = os.path.exists(api_data_path)
print(isExist)

False


In [10]:
if not isExist:
    os.makedirs(api_data_path)

In [18]:
# saving the airport_df to the airport_list.csv
airport_df.to_csv(f'{api_data_path}/airport_list.csv', index=False)

In [7]:
# downloads weather data
def fetch_weather_data():
    start_date = datetime(2019, 1, 1)
    end_date = datetime(2020, 3, 31)
    weather_data = []

    for date in [start_date + relativedelta(months=x) for x in range((end_date.year - start_date.year) * 12 + end_date.month - start_date.month + 1)]:
        response = requests.get(f"{url}/airportWeather?date={date.strftime('%Y-%m')}", headers=headers)
        if response.status_code == 200:
            weather_data.extend(response.json())
        time.sleep(0.1)

    return pd.DataFrame(weather_data)

weather_df = fetch_weather_data()

In [8]:
airport_weather_df = weather_df
airport_weather_df

In [28]:
weather_df.to_csv(f'{api_data_path}/airport_weather.csv', index=False)

In [47]:
# downloads aircraft data
def fetch_aircraft_data():
    response = requests.get(f"{url}/aircraft", headers=headers)
    if response.status_code == 200:
        return pd.DataFrame(response.json())
    return pd.DataFrame()

aircraft_df = fetch_aircraft_data()
aircraft_df

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0
...,...,...,...
7378,2019,N14011,337.0
7379,2019,N16008,337.0
7380,2019,N16009,337.0
7381,2019,N2250U,276.0


In [49]:
aircraft_df.to_csv(f'{api_data_path}/aircraft.csv', index=False)

In [71]:
# downloads flights data
def fetch_flight_data(file_path, url, headers):
    # Wczytywanie pliku CSV
    airports_df = pd.read_csv(file_path)
    airport_ids = airports_df['origin_airport_id'].unique()
    
    # Defining a date range
    start_date = datetime(2019, 1, 1)
    end_date = datetime(2020, 3, 31)
    dates = [start_date + relativedelta(months=mon) for mon in range((end_date.year - start_date.year) * 12 + end_date.month - start_date.month + 1)]
    formatted_dates = [date.strftime('%Y-%m') for date in dates]
    
    # Downloading flight data
    flight_data = []
    for airport_id in airport_ids:
        no_data_found = False
        for date in formatted_dates:
            if no_data_found:
                break
            response = requests.get(f"{url}/flight?airportId={airport_id}&date={date}", headers=headers)
            if response.status_code == 200:
                f_data = response.json()
                flight_data.extend(f_data) 
                print(f"Data downloaded for airport ID {airport_id} for a month {date}.")
            elif response.status_code == 204:
                print(f"No data available for airport ID {airport_id} for a month {date}.")
                no_data_found = True
    
    return pd.DataFrame.from_records(flight_data)

# Path to the CSV file and API connection parameters
file_path = '.data/airports.csv'
url = f'https://api-datalab.coderslab.com/api/v2'
headers = {"authorization": api_key}

flight_df = fetch_flight_data(file_path, url, headers)
flight_df.shape

Brak danych dla lotniska ID 10874 na miesiąc 2019-01.
Brak danych dla lotniska ID 11233 na miesiąc 2019-01.
Brak danych dla lotniska ID 13360 na miesiąc 2019-01.
Brak danych dla lotniska ID 15008 na miesiąc 2019-01.
Brak danych dla lotniska ID 11638 na miesiąc 2019-01.
Brak danych dla lotniska ID 14150 na miesiąc 2019-01.
Brak danych dla lotniska ID 15323 na miesiąc 2019-01.
Brak danych dla lotniska ID 14814 na miesiąc 2019-01.
Brak danych dla lotniska ID 12007 na miesiąc 2019-01.
Brak danych dla lotniska ID 11337 na miesiąc 2019-01.
Brak danych dla lotniska ID 13342 na miesiąc 2019-01.
Brak danych dla lotniska ID 15070 na miesiąc 2019-01.
Brak danych dla lotniska ID 13244 na miesiąc 2019-01.
Brak danych dla lotniska ID 12280 na miesiąc 2019-01.
Brak danych dla lotniska ID 15096 na miesiąc 2019-01.
Brak danych dla lotniska ID 11641 na miesiąc 2019-01.
Brak danych dla lotniska ID 13832 na miesiąc 2019-01.
Brak danych dla lotniska ID 10268 na miesiąc 2019-01.
Dane pobrane dla lotniska ID

Brak danych dla lotniska ID 16869 na miesiąc 2019-01.
Brak danych dla lotniska ID 10408 na miesiąc 2019-01.
Brak danych dla lotniska ID 12264 na miesiąc 2019-01.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-01.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-02.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-03.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-04.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-05.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-06.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-07.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-08.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-09.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-10.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-11.
Dane pobrane dla lotniska ID 11618 na miesiąc 2019-12.
Dane pobrane dla lotniska ID 11618 na miesiąc 2020-01.
Dane pobrane dla lotniska ID 11618 na miesiąc 2020-02.
Dane pobrane dla lotniska ID 11618 na miesiąc 2020-03.
Dane pobrane 

Dane pobrane dla lotniska ID 13204 na miesiąc 2019-01.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-02.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-03.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-04.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-05.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-06.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-07.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-08.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-09.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-10.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-11.
Dane pobrane dla lotniska ID 13204 na miesiąc 2019-12.
Dane pobrane dla lotniska ID 13204 na miesiąc 2020-01.
Dane pobrane dla lotniska ID 13204 na miesiąc 2020-02.
Dane pobrane dla lotniska ID 13204 na miesiąc 2020-03.
Brak danych dla lotniska ID 12992 na miesiąc 2019-01.
Brak danych dla lotniska ID 13184 na miesiąc 2019-01.
Brak danych dla lotniska ID 10643 na miesiąc 2019-01.
Brak danych d

Brak danych dla lotniska ID 11630 na miesiąc 2019-01.
Brak danych dla lotniska ID 10170 na miesiąc 2019-01.
Brak danych dla lotniska ID 12448 na miesiąc 2019-01.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-01.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-02.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-03.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-04.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-05.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-06.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-07.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-08.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-09.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-10.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-11.
Dane pobrane dla lotniska ID 12953 na miesiąc 2019-12.
Dane pobrane dla lotniska ID 12953 na miesiąc 2020-01.
Dane pobrane dla lotniska ID 12953 na miesiąc 2020-02.
Dane pobrane dla lotniska ID 12953 na miesiąc 2020-03.
Brak danych d

Dane pobrane dla lotniska ID 11433 na miesiąc 2019-12.
Dane pobrane dla lotniska ID 11433 na miesiąc 2020-01.
Dane pobrane dla lotniska ID 11433 na miesiąc 2020-02.
Dane pobrane dla lotniska ID 11433 na miesiąc 2020-03.
Brak danych dla lotniska ID 13290 na miesiąc 2019-01.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-01.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-02.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-03.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-04.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-05.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-06.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-07.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-08.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-09.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-10.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-11.
Dane pobrane dla lotniska ID 11298 na miesiąc 2019-12.
Dane pobrane dla lotniska ID 11298 na miesiąc 2020-01.
Dane pobran

Dane pobrane dla lotniska ID 14869 na miesiąc 2019-11.
Dane pobrane dla lotniska ID 14869 na miesiąc 2019-12.
Dane pobrane dla lotniska ID 14869 na miesiąc 2020-01.
Dane pobrane dla lotniska ID 14869 na miesiąc 2020-02.
Dane pobrane dla lotniska ID 14869 na miesiąc 2020-03.
Brak danych dla lotniska ID 14027 na miesiąc 2019-01.
Brak danych dla lotniska ID 14674 na miesiąc 2019-01.
Brak danych dla lotniska ID 12197 na miesiąc 2019-01.
Brak danych dla lotniska ID 14696 na miesiąc 2019-01.
Brak danych dla lotniska ID 13485 na miesiąc 2019-01.
Brak danych dla lotniska ID 10781 na miesiąc 2019-01.
Brak danych dla lotniska ID 10849 na miesiąc 2019-01.
Brak danych dla lotniska ID 14955 na miesiąc 2019-01.
Brak danych dla lotniska ID 13433 na miesiąc 2019-01.
Brak danych dla lotniska ID 15074 na miesiąc 2019-01.
Brak danych dla lotniska ID 13127 na miesiąc 2019-01.
Brak danych dla lotniska ID 11013 na miesiąc 2019-01.
Brak danych dla lotniska ID 14122 na miesiąc 2019-01.
Brak danych dla lotnisk

Dane pobrane dla lotniska ID 13303 na miesiąc 2020-01.
Dane pobrane dla lotniska ID 13303 na miesiąc 2020-02.
Dane pobrane dla lotniska ID 13303 na miesiąc 2020-03.
Brak danych dla lotniska ID 13795 na miesiąc 2019-01.
Brak danych dla lotniska ID 10693 na miesiąc 2019-01.
Brak danych dla lotniska ID 12945 na miesiąc 2019-01.
Brak danych dla lotniska ID 12511 na miesiąc 2019-01.


(1386120, 27)

In [72]:
flight_df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,YEAR,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,20,7,WN,N204WN,682,10397,11292,605,602.0,...,205,204.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
1,1,20,7,WN,N8682B,2622,10397,11292,2120,2114.0,...,210,205.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
2,1,20,7,WN,N717SA,2939,10397,11292,1800,1807.0,...,210,220.0,1199,5,2019,4.0,0.0,10.0,0.0,3.0
3,1,20,7,WN,N709SW,3848,10397,11292,1355,1354.0,...,205,204.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
4,1,20,7,WN,N7864B,1352,10397,11697,1125,1125.0,...,120,124.0,581,3,2019,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386115,3,26,4,DL,N350DN,1982,13303,12953,1956,1944.0,...,183,169.0,1096,5,2020,NaN,NaN,NaN,NaN,NaN
1386116,3,26,4,DL,N908DE,1987,13303,10397,1120,1117.0,...,121,109.0,594,3,2020,NaN,NaN,NaN,NaN,NaN
1386117,3,26,4,DL,NaN,1998,13303,10397,1817,NaN,...,125,NaN,594,3,2020,NaN,NaN,NaN,NaN,NaN
1386118,3,26,4,DL,N352NW,2025,13303,10397,1937,1928.0,...,123,107.0,594,3,2020,NaN,NaN,NaN,NaN,NaN


In [75]:
flight_df.to_csv(f'{api_data_path}/flight.csv', index=False)